# Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# read csv file.
city_data_df = pd.read_csv("weather_data/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Bambous Virieux,MU,-20.34,57.76,72.00,77,75,13.87,shower rain,0.00,0
1,1,Castro,BR,-24.79,-50.01,69.67,63,44,12.01,scattered clouds,0.00,0
2,2,San Fernando,PH,15.03,120.69,80.60,83,75,3.36,broken clouds,0.00,0
3,3,Gbarnga,LR,7.00,-9.47,76.84,78,99,3.69,light rain,0.13,0
4,4,Nanortalik,GL,60.14,-45.24,44.58,92,98,9.37,overcast clouds,0.00,0


In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
rain = input("Do you want it to be raining? (yes/no)")
snow = input("Do you want it to be snowing? (yes/no)")

Do you want it to be raining? (yes/no)yes
Do you want it to be snowing? (yes/no)no


In [5]:
# Filter the DataFrame for minimum and maximum temperature preferences, rain or snowr using conditional statements:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
if rain == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Rain (inches)"]!=0)]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Rain (inches)"]==0)]
if snow == "yes":
     preferred_cities_df = city_data_df.loc[(city_data_df["Snow (inches)"]!=0)]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Snow (inches)"]==0)]
    

In [6]:
#Create DataFrame called hotel_df to store hotel names along with city, country.current description, max temp, and coordinates.
hotel_df = preferred_cities_df[["City","Country","Current Description","Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,shower rain,72.00,-20.34,57.76,
1,Castro,BR,scattered clouds,69.67,-24.79,-50.01,
2,San Fernando,PH,broken clouds,80.60,15.03,120.69,
3,Gbarnga,LR,light rain,76.84,7.00,-9.47,
4,Nanortalik,GL,overcast clouds,44.58,60.14,-45.24,


In [28]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_Vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [7]:
# Set the parameters 
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}

In [8]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
hotel_df.head(10)

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,shower rain,72.00,-20.34,57.76,Casa Tia Villa
1,Castro,BR,scattered clouds,69.67,-24.79,-50.01,CHACARA BAILLY
2,San Fernando,PH,broken clouds,80.60,15.03,120.69,Sky Hotel
3,Gbarnga,LR,light rain,76.84,7.00,-9.47,Passion Hotel 2
4,Nanortalik,GL,overcast clouds,44.58,60.14,-45.24,"NTS Rooms, Nanortalik"
5,Narsaq,GL,broken clouds,55.40,60.92,-46.05,Hotel Narsaq
6,Port Alfred,ZA,light rain,52.00,-33.59,26.89,The Halyards Hotel
7,Hermanus,ZA,scattered clouds,57.00,-34.42,19.23,Misty Waves Boutique Hotel
8,Ushuaia,AR,broken clouds,32.00,-54.80,-68.30,Albatros Hotel
9,Mount Isa,AU,broken clouds,62.60,-20.73,139.50,ibis Styles Mt Isa Verona


In [10]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>  and {Max Temp} °F</dd></dl>

</dl>
"""

In [11]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))